In [1]:
import pandas as pd

In [2]:
labels = ['hatespeech', 'namecalling', 'negprejudice', 'noneng', 'obscenity', 'porn', 'stereotypes', 'threat']

In [3]:
results = pd.read_csv("test_set_results/results_hatespeech.txt", header = None)
results.columns = ['ID', 'prob_hatespeech', 'pred_hatespeech']

for label in labels[1:len(labels)]:
    results_a = pd.read_csv("test_set_results/results_" + label + ".txt", header = None)
    results_a.columns = ['ID', 'prob_' + label, 'pred_' + label]
    results = pd.merge(results, results_a, on = 'ID')

In [4]:
results.head()

,ID,prob_hatespeech,pred_hatespeech,prob_namecalling,pred_namecalling,prob_negprejudice,pred_negprejudice,prob_noneng,pred_noneng,prob_obscenity,pred_obscenity,prob_porn,pred_porn,prob_stereotypes,pred_stereotypes,prob_threat,pred_threat
0,0,0.893101,1,0.660394,1,0.843567,1,0.000336,0,0.967664,1,0.000378,0,0.295742,0,0.001356,0
1,1,0.000793,0,0.012316,0,0.002445,0,0.000349,0,0.000992,0,0.000406,0,0.001943,0,0.000885,0
2,2,0.000706,0,0.000790,0,0.002108,0,0.000301,0,0.000740,0,0.000419,0,0.228456,0,0.000824,0
3,3,0.000797,0,0.000876,0,0.003466,0,0.000283,0,0.000754,0,0.000366,0,0.001292,0,0.000767,0
4,4,0.000750,0,0.023015,0,0.001667,0,0.000323,0,0.000615,0,0.000380,0,0.001276,0,0.000843,0


### Confirm that prevalence of labels roughly matches with training set

In [7]:
for label in labels:
    print (results['pred_' + label].value_counts()/results.shape[0])

0    0.97781
1    0.02219
Name: pred_hatespeech, dtype: float64
0    0.926986
1    0.073014
Name: pred_namecalling, dtype: float64
0    0.886185
1    0.113815
Name: pred_negprejudice, dtype: float64
0    0.922691
1    0.077309
Name: pred_noneng, dtype: float64
0    0.956335
1    0.043665
Name: pred_obscenity, dtype: float64
0    0.992126
1    0.007874
Name: pred_porn, dtype: float64
0    0.937724
1    0.062276
Name: pred_stereotypes, dtype: float64
0    0.967788
1    0.032212
Name: pred_threat, dtype: float64


In [8]:
frame = pd.read_csv("../test_nn.csv")

In [9]:
with_labels = pd.merge(frame, results, on = 'ID')

In [10]:
results.shape

(1397, 17)

In [11]:
frame.shape

(1397, 103)

In [12]:
with_labels.shape

(1397, 119)

### Verifying that class probabilities make intuitive sense

In [13]:
pd.set_option('display.max_colwidth', -1)

In [33]:
with_labels.loc[with_labels['prob_hatespeech'] > 0.50,\
                ['Tweet', 'prob_hatespeech', 'pred_hatespeech']].\
sort_values(by = 'prob_hatespeech')[0:5]

,Tweet,prob_hatespeech,pred_hatespeech
778,Why does #BlackLivesMatter hate Trump? The #1 competition that blacks have in the workforce are immigrants. Are you guys just retarded?,0.505175,1
281,RT @Liberienne: Bout to confront all crackers and ask them to explain the history of the United States today https://t.co/zDB5leJYHJ,0.687190,1
1223,Latinas are so nuts,0.741296,1
728,RT @JohnFromCranber: Carson: USA Must Keep Middle East Refugees Out https://t.co/NwKWSdBXcL … Too Many Bad Apples on Tree of Islam #tcot ht…,0.755345,1
154,RT @Maitre_Eolas: Didn't see any church lit up either. Dirty christians. All terrorists. https://t.co/4s3XCVRbOo,0.784724,1


In [18]:
from sklearn.metrics import f1_score

In [25]:
labels = ['Obscenity', 'Threat', 'hatespeech', 'namecalling', 'negprejudice', 'noneng', 'porn', 'stereotypes']

In [29]:
for label in labels:
    print (label, f1_score(with_labels[label], with_labels['pred_' + label.lower()]))

Obscenity 0.803278688525
Threat 0.4375
hatespeech 0.727272727273
namecalling 0.582914572864
negprejudice 0.529914529915
noneng 0.932126696833
porn 0.761904761905
stereotypes 0.364532019704


In [98]:
terms = pd.read_csv("../terms.csv")

In [99]:
terms = terms[['left-wing ', 'centrist', 'right-wing']]

In [100]:
terms.columns = 'left_wing', 'centrist', 'right_wing'

In [101]:
left = [x.lower().rstrip().lstrip() for x in terms.left_wing]

In [102]:
right = [x.lower().rstrip().lstrip() for x in terms.right_wing[terms.right_wing.isnull() == False]]

In [103]:
center = [x.lower().rstrip().lstrip() for x in terms.centrist[terms.centrist.isnull() == False]]

In [106]:
left_term_found = []

for tweet in with_labels.clean_tweet:
    found = False
    for term in left:
        if term in tweet:
            found = True
    left_term_found.append(found)

In [107]:
right_term_found = []

for tweet in with_labels.clean_tweet:
    found = False
    for term in right:
        if term in tweet:
            found = True
    right_term_found.append(found)

In [108]:
center_term_found = []

for tweet in with_labels.clean_tweet:
    found = False
    for term in center:
        if term in tweet:
            found = True
    center_term_found.append(found)

In [120]:
with_labels.loc[:, 'left'] = left_term_found
with_labels.loc[:, 'right'] = right_term_found
with_labels.loc[:, 'center'] = center_term_found

In [136]:
#only print f-score if there are at least 20 positive examples with the keyword of that category
for label in labels:
    if with_labels.loc[with_labels.left + with_labels[label] == 2, :].shape[0] > 20:
        print (label, 'left', f1_score(with_labels.loc[with_labels.left == True, label], \
                                   with_labels.loc[with_labels.left == True, 'pred_' + label.lower()]))
    if with_labels.loc[with_labels.right + with_labels[label] == 2, :].shape[0] > 20:
        print (label, 'center', f1_score(with_labels.loc[with_labels.center == True, label], \
                                   with_labels.loc[with_labels.center == True, 'pred_' + label.lower()]))
    if with_labels.loc[with_labels.right + with_labels[label] == 2, :].shape[0] > 20:
        print (label, 'right', f1_score(with_labels.loc[with_labels.right == True, label], \
                                   with_labels.loc[with_labels.right == True, 'pred_' + label.lower()]))

Obscenity left 0.83950617284
Threat left 0.510638297872
hatespeech left 0.793103448276
namecalling left 0.6
namecalling center 0.833333333333
namecalling right 0.491803278689
negprejudice left 0.530120481928
negprejudice center 0.75
negprejudice right 0.543859649123
noneng center 0.909090909091
noneng right 0.872727272727
stereotypes left 0.32183908046
stereotypes center 0.533333333333
stereotypes right 0.38961038961


### Writing results to file

In [ ]:
#with_labels.to_csv("../all_labeled_tweets.csv")